In [1]:
%matplotlib qt
import pyxem as pxm
import hyperspy.api as hs
import os
import gc
import numpy as np
from tempfile import TemporaryFile
import tqdm

In [2]:
data = []
for root, dirs, files in os.walk(r"D:\jf631\mg24111-1"):
    for file in files:
        if file.endswith('nonthreshold_radial.hspy'):# and not file.endswith('._radial_integration.hspy'):
            data.append(os.path.join(root, file))
data.sort()

In [3]:
data[:3]

['D:\\jf631\\mg24111-1\\20200927_094445_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_253x255_diff_plane_515x515_nonthreshold_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_101523_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_254x255_diff_plane_515x515_nonthreshold_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_101748_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_255x255_diff_plane_515x515_nonthreshold_radial.hspy']

In [4]:
start = 0.11
stop = 1.30
signal_len = 182
sqrt = False

for fname in tqdm.tqdm_notebook(data[:]):
    s = hs.load(fname)
    s.crop_signal1D(start, stop)
    s = s.rebin([s.data.shape[1], s.data.shape[0], signal_len])
    if sqrt:
        s.data = np.sqrt(s.data)
    dpmax = s.data.max(2)
    s_norm = s.data/dpmax[:,:,np.newaxis]
    s_norm = pxm.ElectronDiffraction1D(s_norm)
    
    experimental_normalised_cropped = TemporaryFile()
    exp1d = s_norm.data
    if sqrt:
        path = os.path.join(os.path.dirname(fname), '{}_norm_cropped_sqrt'.format(os.path.basename(fname)))
    else:
        path = os.path.join(os.path.dirname(fname), '{}_norm_cropped'.format(os.path.basename(fname)))
    np.savez(path, exp1d=exp1d)
    
    del s
    del s_norm
    gc.collect()

C:\Users\Sauron\anaconda3\envs\pyxem\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


## Parallelization

In [29]:
start = 0.11
stop = 1.30
signal_len = 182
sqrt = False


def normalise_crop_save_in_parallel(fname_path, crop_start_q, crop_stop_q, cropped_signal_len):
    
    def normalize(dp):
        dpmax = dp.data.max(2)
        dp_norm = dp.data / dpmax[:,:,np.newaxis]
        dp_norm = pxm.ElectronDiffraction1D(dp_norm)
        return dp_norm
    
    import pyxem as pxm
    import numpy as np
    import os
    import gc
    from tempfile import TemporaryFile
    
    folder = os.path.dirname(fname_path)
    fname = os.path.basename(fname_path).split(".")[0]
    
    s = pxm.load(fname_path)
    s.crop_signal1D(crop_start_q, crop_stop_q)
    s = s.rebin([s.data.shape[1], s.data.shape[0], cropped_signal_len])
    
    s_sqrt = s.deepcopy()
    s_sqrt.data = np.sqrt(s_sqrt.data)
    
    s_norm = normalize(s)
    s_norm_sqrt = normalize(s_sqrt)
    
    experimental_normalised_cropped = TemporaryFile()
    path = os.path.join(folder, '{}_norm_cropped'.format(fname))
    np.savez(path, exp1d=s_norm)
    
    experimental_normalised_cropped_sqrt = TemporaryFile()
    path = os.path.join(folder, '{}_norm_cropped_sqrt'.format(fname))
    np.savez(path, exp1d=s_norm_sqrt)
    
    del s
    del s_norm
    del s_sqrt
    del s_norm_sqrt
    gc.collect()
    return

## Thresholded data

In [30]:
# Define parameters for thresholded data
import os
import numpy as np
from itertools import product

start = 0.11
stop = 1.30
signal_len = 182
str_ends_with = '_centred_thresholded_radial.hspy'

fnames = []
for root, dirs, files in os.walk(r"D:\jf631\mg24111-1"):
    for file in files:
        if file.endswith(str_ends_with):
            fnames.append(os.path.join(root, file))
fnames.sort()

# Create iteration tools
start = [start,]
stop = [stop,]
signal_len = [signal_len,]

iterations = product(fnames, start, stop, signal_len)

In [31]:
fnames[4:8]

['D:\\jf631\\mg24111-1\\20200927_102109_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_255x255_diff_plane_515x515_centred_thresholded_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_102301_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_255x255_diff_plane_515x515_centred_thresholded_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_102434_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_255x255_diff_plane_515x515_centred_thresholded_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_102636_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_255x255_diff_plane_515x515_centred_thresholded_radial.hspy']

In [32]:
# Run
# make sure to always use multiprocess
from multiprocess import Pool
import psutil

# start your parallel workers at the beginning of your script
n_cores = psutil.cpu_count(logical=False)
pool = Pool(n_cores)

# execute a computation(s) in parallel
pool.starmap(normalise_crop_save_in_parallel, iterations)

# turn off your parallel workers at the end of your script
pool.close()

## Non-thresholded data

In [33]:
# Define parameters for thresholded data
import os
import numpy as np
from itertools import product

start = 0.11
stop = 1.30
signal_len = 182
str_ends_with = '_centred_nonthresholded_radial.hspy'

fnames = []
for root, dirs, files in os.walk(r"D:\jf631\mg24111-1"):
    for file in files:
        if file.endswith(str_ends_with):
            fnames.append(os.path.join(root, file))
fnames.sort()

# Create iteration tools
start = [start,]
stop = [stop,]
signal_len = [signal_len,]

iterations = product(fnames, start, stop, signal_len)

In [34]:
fnames[:3]

['D:\\jf631\\mg24111-1\\20200927_094445_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_253x255_diff_plane_515x515_centred_nonthresholded_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_101523_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_254x255_diff_plane_515x515_centred_nonthresholded_radial.hspy',
 'D:\\jf631\\mg24111-1\\20200927_101748_TripCatBeamDam_3p59A2_150kx_10umAp_20cmCL_scan_array_255x255_diff_plane_515x515_centred_nonthresholded_radial.hspy']

In [35]:
# Run
# make sure to always use multiprocess
from multiprocess import Pool
import psutil

# start your parallel workers at the beginning of your script
n_cores = psutil.cpu_count(logical=False)
pool = Pool(n_cores)

# execute a computation(s) in parallel
pool.starmap(normalise_crop_save_in_parallel, iterations)

# turn off your parallel workers at the end of your script
pool.close()